# 📊 Results Visualization

This notebook visualizes the experiment results with publication-ready figures.

---

In [ ]:
# ============================================================
# SETUP
# ============================================================

import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path
import os

# Style settings for publication
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['font.size'] = 12
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.titlesize'] = 16
plt.rcParams['legend.fontsize'] = 11
plt.rcParams['figure.dpi'] = 100

# Find project root
notebook_dir = Path(os.getcwd())
project_root = notebook_dir.parent if 'notebooks' in str(notebook_dir) else notebook_dir
figures_dir = project_root / 'results' / 'figures'
figures_dir.mkdir(parents=True, exist_ok=True)

print("✅ Setup complete!")
print(f"📁 Figures will be saved to: {figures_dir}")

---

## 📝 Enter Your Results

Fill in the results from your experiments below.

In [ ]:
# ============================================================
# ENTER YOUR RESULTS HERE
# ============================================================

# Experiment 01: Baseline Gap
BASELINE_LAB_ACC = 99.36       # Lab accuracy (%)
BASELINE_FIELD_ACC = 26.17    # Field accuracy (%)

# Experiment 02: Passive Augmentation
PASSIVE_AUG_LAB_ACC = 90.12
PASSIVE_AUG_FIELD_ACC = 32.21

# Experiment 03: CutMix
CUTMIX_LAB_ACC = 88.45
CUTMIX_FIELD_ACC = 35.57

# Experiment 04 & 05: Active Learning Results
# x-axis: number of labeled field images
AL_X_VALUES = [0, 50, 100, 150, 200]

# Accuracies at each point
AL_RANDOM = [26.17, 25.50, 33.56, 41.61, 44.97]
AL_ENTROPY = [26.17, 26.17, 28.19, 40.27, 42.95]
AL_HYBRID = [26.17, 26.17, 34.23, 38.93, 46.31]

print("✅ Results loaded!")

---

## 📈 Figure 1: Generalization Gap

In [ ]:
# ============================================================
# FIGURE 1: Generalization Gap Visualization
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

methods = ['Baseline', 'Passive Aug.', 'CutMix']
lab_accs = [BASELINE_LAB_ACC, PASSIVE_AUG_LAB_ACC, CUTMIX_LAB_ACC]
field_accs = [BASELINE_FIELD_ACC, PASSIVE_AUG_FIELD_ACC, CUTMIX_FIELD_ACC]
gaps = [lab - field for lab, field in zip(lab_accs, field_accs)]

x = np.arange(len(methods))
width = 0.35

bars1 = ax.bar(x - width/2, lab_accs, width, label='Lab (pv)', color='#27ae60', edgecolor='black')
bars2 = ax.bar(x + width/2, field_accs, width, label='Field (PlantDoc)', color='#e74c3c', edgecolor='black')

# Add gap annotations
for i, (lab, field, gap) in enumerate(zip(lab_accs, field_accs, gaps)):
    mid_y = (lab + field) / 2
    ax.annotate('', xy=(i + width/2, field), xytext=(i - width/2, lab),
                arrowprops=dict(arrowstyle='<->', color='#2c3e50', lw=2))
    ax.text(i, mid_y, f'Gap: {gap:.1f}%', ha='center', va='center', 
            fontsize=10, fontweight='bold', color='#2c3e50',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

ax.set_ylabel('Accuracy (%)')
ax.set_title('Generalization Gap: Lab vs. Field Accuracy', fontweight='bold')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend(loc='upper right')
ax.set_ylim(0, 110)

# Add value labels on bars
for bar in bars1:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)
for bar in bars2:
    height = bar.get_height()
    ax.annotate(f'{height:.1f}%', xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 3), textcoords="offset points", ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.savefig(figures_dir / 'Figure1_Generalization_Gap.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_dir / 'Figure1_Generalization_Gap.pdf', bbox_inches='tight')
plt.show()

print(f"\n✅ Saved to {figures_dir / 'Figure1_Generalization_Gap.png'}")

---

## 📈 Figure 2: Active Learning Comparison (Main Result)

In [ ]:
# ============================================================
# FIGURE 2: Active Learning Comparison (Main Paper Figure)
# ============================================================

fig, ax = plt.subplots(figsize=(10, 6))

# Plot lines
ax.plot(AL_X_VALUES, AL_RANDOM, 'o-', label='Random Sampling', 
        color='gray', linewidth=2, markersize=8, alpha=0.7)
ax.plot(AL_X_VALUES, AL_ENTROPY, 's-', label='Entropy Sampling', 
        color='#f39c12', linewidth=2, markersize=8)
ax.plot(AL_X_VALUES, AL_HYBRID, '^-', label='Hybrid (Ours)', 
        color='#e74c3c', linewidth=3, markersize=10)

# Highlight final values
for i, (r, e, h) in enumerate(zip(AL_RANDOM, AL_ENTROPY, AL_HYBRID)):
    if i == len(AL_X_VALUES) - 1:  # Last point
        ax.annotate(f'{r:.1f}%', (AL_X_VALUES[i], r), textcoords="offset points",
                    xytext=(5, -15), fontsize=9, color='gray')
        ax.annotate(f'{e:.1f}%', (AL_X_VALUES[i], e), textcoords="offset points",
                    xytext=(5, 5), fontsize=9, color='#f39c12')
        ax.annotate(f'{h:.1f}%', (AL_X_VALUES[i], h), textcoords="offset points",
                    xytext=(5, 5), fontsize=10, fontweight='bold', color='#e74c3c')

# Add baseline reference line
ax.axhline(y=BASELINE_FIELD_ACC, color='lightgray', linestyle='--', 
           label=f'Baseline ({BASELINE_FIELD_ACC:.1f}%)', alpha=0.7)

ax.set_xlabel('Number of Labeled Field Images')
ax.set_ylabel('Field Test Accuracy (%)')
ax.set_title('Active Learning Strategies Comparison', fontweight='bold')
ax.legend(loc='lower right')
ax.set_xlim(-10, max(AL_X_VALUES) + 20)
ax.set_ylim(20, 55)
ax.grid(True, alpha=0.3)

# Add annotation for the "dip"
ax.annotate('Entropy "dip"', xy=(100, AL_ENTROPY[2]), 
            xytext=(130, AL_ENTROPY[2]-5),
            arrowprops=dict(arrowstyle='->', color='#f39c12'),
            fontsize=9, color='#f39c12')

plt.tight_layout()
plt.savefig(figures_dir / 'Figure2_Active_Learning.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_dir / 'Figure2_Active_Learning.pdf', bbox_inches='tight')
plt.show()

print(f"\n✅ Saved to {figures_dir / 'Figure2_Active_Learning.png'}")

---

## 📈 Figure 3: Combined Overview

In [ ]:
# ============================================================
# FIGURE 3: Combined Overview (Two-panel figure)
# ============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Panel A: Gap comparison
methods = ['Baseline', 'Passive Aug.', 'CutMix']
gaps = [BASELINE_LAB_ACC - BASELINE_FIELD_ACC, 
        PASSIVE_AUG_LAB_ACC - PASSIVE_AUG_FIELD_ACC,
        CUTMIX_LAB_ACC - CUTMIX_FIELD_ACC]
field_accs = [BASELINE_FIELD_ACC, PASSIVE_AUG_FIELD_ACC, CUTMIX_FIELD_ACC]

colors = ['#e74c3c', '#f39c12', '#3498db']
bars = ax1.bar(methods, field_accs, color=colors, edgecolor='black', alpha=0.8)

ax1.set_ylabel('Field Accuracy (%)')
ax1.set_title('(a) Passive Methods', fontweight='bold')
ax1.set_ylim(0, 50)

for bar, gap in zip(bars, gaps):
    height = bar.get_height()
    ax1.annotate(f'{height:.1f}%\n(gap: {gap:.1f}%)', 
                xy=(bar.get_x() + bar.get_width()/2, height),
                xytext=(0, 5), textcoords="offset points", 
                ha='center', va='bottom', fontsize=10)

# Panel B: Active learning
ax2.plot(AL_X_VALUES, AL_RANDOM, 'o-', label='Random', color='gray', linewidth=2, markersize=8)
ax2.plot(AL_X_VALUES, AL_ENTROPY, 's-', label='Entropy', color='#f39c12', linewidth=2, markersize=8)
ax2.plot(AL_X_VALUES, AL_HYBRID, '^-', label='Hybrid (Ours)', color='#e74c3c', linewidth=3, markersize=10)

ax2.set_xlabel('Labeled Field Images')
ax2.set_ylabel('Field Accuracy (%)')
ax2.set_title('(b) Active Learning Strategies', fontweight='bold')
ax2.legend(loc='lower right')
ax2.grid(True, alpha=0.3)
ax2.set_ylim(20, 55)

plt.tight_layout()
plt.savefig(figures_dir / 'Figure3_Combined_Overview.png', dpi=300, bbox_inches='tight')
plt.savefig(figures_dir / 'Figure3_Combined_Overview.pdf', bbox_inches='tight')
plt.show()

print(f"\n✅ Saved to {figures_dir / 'Figure3_Combined_Overview.png'}")

---

## 📊 Results Table

In [ ]:
# ============================================================
# RESULTS TABLE
# ============================================================

print("\n" + "="*70)
print("SUMMARY OF RESULTS")
print("="*70)

print("\n📊 PASSIVE METHODS (No field data):")
print("-"*50)
print(f"{'Method':<20} | {'Lab Acc':>10} | {'Field Acc':>10} | {'Gap':>8}")
print("-"*50)
print(f"{'Baseline':<20} | {BASELINE_LAB_ACC:>9.2f}% | {BASELINE_FIELD_ACC:>9.2f}% | {BASELINE_LAB_ACC - BASELINE_FIELD_ACC:>7.2f}%")
print(f"{'Passive Augmentation':<20} | {PASSIVE_AUG_LAB_ACC:>9.2f}% | {PASSIVE_AUG_FIELD_ACC:>9.2f}% | {PASSIVE_AUG_LAB_ACC - PASSIVE_AUG_FIELD_ACC:>7.2f}%")
print(f"{'CutMix':<20} | {CUTMIX_LAB_ACC:>9.2f}% | {CUTMIX_FIELD_ACC:>9.2f}% | {CUTMIX_LAB_ACC - CUTMIX_FIELD_ACC:>7.2f}%")

print("\n📊 ACTIVE LEARNING (With labeling budget):")
print("-"*60)
print(f"{'Labels':<10} | {'Random':>12} | {'Entropy':>12} | {'Hybrid':>12}")
print("-"*60)
for i, x in enumerate(AL_X_VALUES):
    print(f"{x:<10} | {AL_RANDOM[i]:>11.2f}% | {AL_ENTROPY[i]:>11.2f}% | {AL_HYBRID[i]:>11.2f}%")

print("\n" + "="*70)
print("KEY FINDINGS:")
print("="*70)
print(f"1. Baseline gap: {BASELINE_LAB_ACC - BASELINE_FIELD_ACC:.1f}% (Lab→Field accuracy drop)")
print(f"2. Passive methods reduce gap but don't solve the problem")
print(f"3. With {AL_X_VALUES[-1]} labeled field images:")
print(f"   - Random: {AL_RANDOM[-1]:.2f}%")
print(f"   - Entropy: {AL_ENTROPY[-1]:.2f}%")
print(f"   - Hybrid (Ours): {AL_HYBRID[-1]:.2f}% ⭐ BEST")
print(f"4. Hybrid improvement over Random: +{AL_HYBRID[-1] - AL_RANDOM[-1]:.2f}%")
print("="*70)

---

## ✅ Figures Saved!

All figures are saved in:
- `results/figures/Figure1_Generalization_Gap.png`
- `results/figures/Figure2_Active_Learning.png`
- `results/figures/Figure3_Combined_Overview.png`

PDF versions are also saved for LaTeX documents.